# Torrent To Google Drive Downloader 

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [ ]:
import os
!apt install python3-libtorrent
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar9
Suggested packages:
  libtorrent-rasterbar-dbg
The following NEW packages will be installed:
  libtorrent-rasterbar9 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,569 kB of archives.
After this operation, 5,718 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtorrent-rasterbar9 amd64 1.1.5-1build1 [1,258 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-libtorrent amd64 1.1.5-1build1 [310 kB]
Fetched 1,569 kB in 1s (1,601 kB/s)
Selecting previously unselected package libtorrent-rasterbar9.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar9_1.1.5-1build1_amd64.deb ...
Unpacking libtorrent-rasterbar9 (1.1.5-1build1) ...
Se

#Creating a Directory

In [ ]:
try: 
    os.mkdir('/content/Torrent/') 
    print("Directory 'Torrent/' created") 
except OSError as error: 
    print(error)

Directory 'Torrent/' created


### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [ ]:
params = {
    "save_path": "/content/Torrent"
    #"save_path": "/content/drive/MyDrive/Torrent"
    }

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit" or magnet_link.lower()=="":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:89925FB48CAE260801F35FB7175530BF6E5E055A&dn=Captain+America%3A+Civil+War+%282016%29+English+BrRip+1080p+x264&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2710%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969&tr=udp%3A%2F%2Ftracker.internetwarriors.net%3A1337&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fcoppersurfer.tk%3A6969%2Fannounce
Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:414A6F933C48FC7543A9CDB42C854B5457C5BCC7&dn=Avengers%3A+Endgame+%282019%29+%5BWEBRip%5D+%5B1080p%5D+%5BYTS%5D+%5BYIFY%5D&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2710%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337&tr=udp%3A%2F%2Ftracker.internetwarriors.net%3A1337&tr=udp%3A%2F

### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files

source = files.upload()
params = {
    #"save_path": "/content/drive/MyDrive/Torrent",
    "save_path": "/content/Torrent",

    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
Flag = True

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
            flag = False
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM] complete
Captain America Civil War (2016) [1080p] [YTS.AG] complete
Avengers Endgame (2019) [WEBRip] [1080p] [YTS.LT] complete


# Zip/Unzip

In [ ]:
!zip -r /content/Torrent/13.Re.zip /content/Torrent/13.Re

In [ ]:
!unzip path_to_file.zip -d path_to_directory

### Mount Google Drive
To stream files we need to mount Google Drive.

In [ ]:
from google.colab import drive

In [ ]:
#Mount
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#Unmount
drive.flush_and_unmount()

#Copy to drive

In [ ]:
#path that contains folder you want to copy
#%cd /gdrive/My Drive/...
#%cp -av YOUR_FOLDER NEW_FOLDER_COPY
%cd /content/drive/MyDrive/Torrent
%cp -av /content/Torrent/W10p W10p

/content/drive/MyDrive/Torrent
cp: cannot stat '/content/Torrent/W10p': No such file or directory


#Copy files from one Folder to Another.

In [ ]:
#cp -vr dir_A dir_A_copy
%cp -vr /content/Torrent/ /content/drive/MyDrive/

'/content/Torrent/Captain America Civil War (2016) [1080p] [YTS.AG]/WWW.YTS.AG.jpg' -> '/content/drive/MyDrive/Torrent/Captain America Civil War (2016) [1080p] [YTS.AG]/WWW.YTS.AG.jpg'
'/content/Torrent/Avengers Endgame (2019) [WEBRip] [1080p] [YTS.LT]/www.YTS.LT.jpg' -> '/content/drive/MyDrive/Torrent/Avengers Endgame (2019) [WEBRip] [1080p] [YTS.LT]/www.YTS.LT.jpg'
'/content/Torrent/Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM]/Avengers.Infinity.War.2018.1080p.WEBRip.x264-[YTS.AM].mp4' -> '/content/drive/MyDrive/Torrent/Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM]/Avengers.Infinity.War.2018.1080p.WEBRip.x264-[YTS.AM].mp4'
'/content/Torrent/Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM]/www.YTS.AM.jpg' -> '/content/drive/MyDrive/Torrent/Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM]/www.YTS.AM.jpg'
cp: cannot create regular file '/content/drive/MyDrive/Torrent/Avengers Infinity War (2018) [WEBRip] [1080p] [YTS.AM]/www.YTS.AM.jpg': No such file or dir